<div style="text-align: center; font-family: 'charter bt pro roman'; color: rgb(0, 65, 75);">
    <h1>
    GDP Vintages, Releases and Revisions datasets
    </h1>
</div>

<div style="text-align: center; font-family: 'charter bt pro roman'; color: rgb(0, 65, 75);">
<h3>
Documentation
<br>
____________________
<br>
</h3>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    This 
    <span style="color: rgb(0, 65, 75);">jupyter notebook</span>
    provides a step-by-step guide to <b>data building</b> regarding the project <b>'Revisiones y sesgos en las estimaciones preliminares del PBI en el Perú'</b>. This guide covers the creation of GDP mid-term revision dataset for each sector. A key step is the construction at par of what we will call “The ‘t+h’ structure”. This dataset is similar to that of the GDP growth vintages by sector, but instead of growth rate values, it contains values of type “t+h”, where h indicates how many months have passed since the preliminary growth rate was first published; that is, this jupyter notebook also covers the creation of vintages datasets of growth rates associated with a horizon (<b>h</b>).
</div>

<div style="text-align: center; font-family: 'PT Serif Pro Book'; color: rgb(0, 65, 75); font-size: 16px;">
    Jason Cruz
    <br>
    <a href="mailto:jj.cruza@up.edu.pe" style="color: rgb(0, 153, 123); font-size: 16px;">
        jj.cruza@up.edu.pe
    </a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;line-height: 1.5;">
<span style="font-size: 34px;">&#128452;</span> The 't+h' structure should be available for all sectors and frequencies.
    <br>
    <span style="font-size: 24px;">&#8987;</span> Available since <b>1994-2024</b> (Table 1) and since <b>1997-2024</b> (Table 2). 
    <br>
</div>

<div style="font-family: Amaya; text-align: left; color: rgb(0, 65, 75); font-size:16px">The following <b>outline is functional</b>. By utilising the provided buttons, users are able to enhance their experience by browsing this script.<div/>

<div id="outilne">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #292929; padding: 10px; line-height: 1.5; font-family: 'PT Serif Pro Book';">
    <h2 style="text-align: left; color: #E0E0E0;">
        Outline
    </h2>
    <br>
    <a href="#libraries" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        Libraries</a>
    <br>
    <a href="#setup" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        Initial set-up</a>
    <br>
    <a href="#1" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        1. Economic sector selector</a>
    <br>
    <a href="#2" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        2. Create horizon datasets</a>
    <br>
    <a href="#2.1." style="color: #94FFD8; font-size: 16px; margin-left: 20px;">
        2.1. Loading growth rate datasets from postgresql.</a>
    <br>
    <a href="#2.2." style="color: #94FFD8; font-size: 16px; margin-left: 20px;">
        2.2. Creating horizon dataset step by step.</a> 
    <br>
    <a href="#3" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        3. Create growth rates by horizon dataset</a>
    <br>
    <a href="#4" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        4. Create vintages and releases datasets</a>
    <br>
    <a href="#5" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        5. Loading to SQL</a>
    <br>
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Any questions or issues regarding the coding, please email Jason Cruz <a href="mailto:jj.cruza@alum.up.edu.pe" style="color: rgb(0, 153, 123); text-decoration: none;"><span style="font-size: 24px;">&#x2709;</span>
    </a>.
    <div/>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    If you don't have the libraries below, please use the following code (as example) to install the required libraries.
    <div/>

In [ ]:
#!pip install os # Comment this code with "#" if you have already installed this library.

<div id="libraries">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark;">
    <h2>
    Libraries
    </h2>
    <div/>

In [1]:
# POSTGRESSQL
import os
from sqlalchemy import create_engine

# HORIZON DATASETS
import pandas as pd
import numpy as np
import re


<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="setup">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark;">
    <h2>
    Initial set-up
    </h2>
    <div/>

<p style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> The following function will establish a connection to the <code>gdp_revisions_datasets</code> database in <code>PostgreSQL</code>. The <b>input data</b> used in this jupyter notebook will be loaded from this <code>PostgreSQL</code> database, and similarly, all <b>output data</b> generated by this jupyter notebook will be stored in that database. Ensure that you set the necessary parameters to access the server once you have obtained the required permissions.<p/>
    
<p style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
To request permissions, please email Jason Cruz <a href="mailto:jj.cruza@alum.up.edu.pe" style="color: rgb(0, 153, 123); text-decoration: none;"> <span style="font-size: 24px;">&#x2709;</span>
    </a>.
<p/>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    <span style="font-size: 24px; color: #FFA823; font-weight: bold;">&#9888;</span>
    Enter your user credentials to acces to SQL.
    <div/>

In [2]:
def create_sqlalchemy_engine():
    """
    Function to create an SQLAlchemy engine using environment variables.
    
    Returns:
        engine: SQLAlchemy engine object.
    """
    # Get environment variables
    user = os.environ.get('CIUP_SQL_USER')  # Get the SQL user from environment variables
    password = os.environ.get('CIUP_SQL_PASS')  # Get the SQL password from environment variables
    host = os.environ.get('CIUP_SQL_HOST')  # Get the SQL host from environment variables
    port = 5432  # Set the SQL port to 5432
    database = 'gdp_revisions_datasets'  # Set the database name 'gdp_revisions_datasets' from SQL

    # Check if all environment variables are defined
    if not all([host, user, password]):
        raise ValueError("Some environment variables are missing (CIUP_SQL_HOST, CIUP_SQL_USER, CIUP_SQL_PASS)")

    # Create connection string
    connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

    # Create SQLAlchemy engine
    engine = create_engine(connection_string)
    
    return engine

<div style="text-align: left;">
    <span style="font-size: 24px; color: rgb(255, 32, 78); font-weight: bold;">&#9888;</span>
    <span style="font-family: PT Serif Pro Book; color: black; font-size: 16px;">
        Import all other functions required by this jupyter notebook.
    </span>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> Please, check the script <code>gdp_revisions_datasets_functions.py</code> which contains all the functions required by this jupyter notebook. The functions there are ordered according to the <a href="#outilne" style="color: #3d30a2;">sections</a> of this jupyter notebok.<div/>

In [3]:
from gdp_revisions_datasets_functions import *

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="1">
   <!-- Contenido de la celda de destino -->
</div>

<h1><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book; color: dark;">1.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Economic sector and data frequency selector</span></h1>

<div id="steps-1">
   <!-- Contenido de la celda de destino -->
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <a href="#step-1-1" style="text-decoration: none; color: #006769"> <span style="font-size: 24px; color: rgb(0, 65, 75);">&#10122;</span> Select economic sector</a>
    <br>
    <a href="#step-1-2" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> Select frequency</a>
</div>

<div id="step-1-1">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10122;</span> <span>Select economic sector</span>
  </div>

In [4]:
# Call the function to show the popup window
sector = show_option_window()
print("Selected economic sector:", sector)

Selected economic sector: gdp


<div id="step-1-2">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> <span>Select frequency</span>
  </div>

In [5]:
# Call the function to show the popup window
frequency = show_frequency_window()
print("Selected frequency:", frequency)

Selected frequency: monthly


<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-1" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-1" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="2">
   <!-- Contenido de la celda de destino -->
</div>

<h1><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book; color: dark;">2.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Create horizon datasets</span></h1>

<div id="2.1.">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">2.1. </span> <span style = "color: dark; font-family: PT Serif Pro Book;">Loading growth rate datasets from <code>PostgresSQL</code></span></h2>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Connect to SQL.
    <div/>

In [6]:
# Connect to SQL
engine = create_sqlalchemy_engine()

# SQL Query
query = f"SELECT * FROM {sector}_{frequency}_growth_rates;" # Please change your query to PosgtresSQL as you see fit

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Comment the code below if you want the default option (display rows and columns of the dataframe in a limited way)
    <div/>

In [7]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Check the loaded dataframe.
    <div/>

In [8]:
# Read growth rates dataset as DataFrame
globals()[f'{sector}_{frequency}_growth_rates'] = pd.read_sql(query, engine)
growth_rates = globals()[f'{sector}_{frequency}_growth_rates']
growth_rates_df = growth_rates.copy()
growth_rates_df.head(150)

,year,id_ns,date,ene_1992,feb_1992,mar_1992,abr_1992,may_1992,jun_1992,jul_1992,ago_1992,sep_1992,oct_1992,nov_1992,dic_1992,ene_1993,feb_1993,mar_1993,abr_1993,may_1993,jun_1993,jul_1993,ago_1993,sep_1993,oct_1993,nov_1993,dic_1993,ene_1994,feb_1994,mar_1994,abr_1994,may_1994,jun_1994,jul_1994,ago_1994,sep_1994,oct_1994,nov_1994,dic_1994,ene_1995,feb_1995,mar_1995,abr_1995,may_1995,jun_1995,jul_1995,ago_1995,sep_1995,oct_1995,nov_1995,dic_1995,ene_1996,feb_1996,mar_1996,abr_1996,may_1996,jun_1996,jul_1996,ago_1996,sep_1996,oct_1996,nov_1996,dic_1996,ene_1997,feb_1997,mar_1997,abr_1997,may_1997,jun_1997,jul_1997,ago_1997,sep_1997,oct_1997,nov_1997,dic_1997,ene_1998,feb_1998,mar_1998,abr_1998,may_1998,jun_1998,jul_1998,ago_1998,sep_1998,oct_1998,nov_1998,dic_1998,ene_1999,feb_1999,mar_1999,abr_1999,may_1999,jun_1999,jul_1999,ago_1999,sep_1999,oct_1999,nov_1999,dic_1999,ene_2000,feb_2000,mar_2000,abr_2000,may_2000,jun_2000,jul_2000,ago_2000,sep_2000,oct_2000,nov_2000,dic_2000,ene_2001,feb_2001,mar_2001,abr_2001,may_2001,jun_2001,jul_2001,ago_2001,sep_2001,oct_2001,nov_2001,dic_2001,ene_2002,feb_2002,mar_2002,abr_2002,may_2002,jun_2002,jul_2002,ago_2002,sep_2002,oct_2002,nov_2002,dic_2002,ene_2003,feb_2003,mar_2003,abr_2003,may_2003,jun_2003,jul_2003,ago_2003,sep_2003,oct_2003,nov_2003,dic_2003,ene_2004,feb_2004,mar_2004,abr_2004,may_2004,jun_2004,jul_2004,ago_2004,sep_2004,oct_2004,nov_2004,dic_2004,ene_2005,feb_2005,mar_2005,abr_2005,may_2005,jun_2005,jul_2005,ago_2005,sep_2005,oct_2005,nov_2005,dic_2005,ene_2006,feb_2006,mar_2006,abr_2006,may_2006,jun_2006,jul_2006,ago_2006,sep_2006,oct_2006,nov_2006,dic_2006,ene_2007,feb_2007,mar_2007,abr_2007,may_2007,jun_2007,jul_2007,ago_2007,sep_2007,oct_2007,nov_2007,dic_2007,ene_2008,feb_2008,mar_2008,abr_2008,may_2008,jun_2008,jul_2008,ago_2008,sep_2008,oct_2008,nov_2008,dic_2008,ene_2009,feb_2009,mar_2009,abr_2009,may_2009,jun_2009,jul_2009,ago_2009,sep_2009,oct_2009,nov_2009,dic_2009,ene_2010,feb_2010,mar_2010,abr_2010,may_2010,jun_2010,jul_2010,ago_2010,sep_2010,oct_2010,nov_2010,dic_2010,ene_2011,feb_2011,mar_2011,abr_2011,may_2011,jun_2011,jul_2011,ago_2011,sep_2011,oct_2011,nov_2011,dic_2011,ene_2012,feb_2012,mar_2012,abr_2012,may_2012,jun_2012,jul_2012,ago_2012,sep_2012,oct_2012,nov_2012,dic_2012,ene_2013,feb_2013,mar_2013,abr_2013,may_2013,jun_2013,jul_2013,ago_2013,sep_2013,oct_2013,nov_2013,dic_2013,ene_2014,feb_2014,mar_2014,abr_2014,may_2014,jun_2014,jul_2014,ago_2014,sep_2014,oct_2014,nov_2014,dic_2014,ene_2015,feb_2015,mar_2015,abr_2015,may_2015,jun_2015,jul_2015,ago_2015,sep_2015,oct_2015,nov_2015,dic_2015,ene_2016,feb_2016,mar_2016,abr_2016,may_2016,jun_2016,jul_2016,ago_2016,sep_2016,oct_2016,nov_2016,dic_2016,ene_2017,feb_2017,mar_2017,abr_2017,may_2017,jun_2017,jul_2017,ago_2017,sep_2017,oct_2017,nov_2017,dic_2017,ene_2018,feb_2018,mar_2018,abr_2018,may_2018,jun_2018,jul_2018,ago_2018,sep_2018,oct_2018,nov_2018,dic_2018,ene_2019,feb_2019,mar_2019,abr_2019,may_2019,jun_2019,jul_2019,ago_2019,sep_2019,oct_2019,nov_2019,dic_2019,ene_2020,feb_2020,mar_2020,abr_2020,may_2020,jun_2020,jul_2020,ago_2020,sep_2020,oct_2020,nov_2020,dic_2020,ene_2021,feb_2021,mar_2021,abr_2021,may_2021,jun_2021,jul_2021,ago_2021,sep_2021,oct_2021,nov_2021,dic_2021,ene_2022,feb_2022,mar_2022,abr_2022,may_2022,jun_2022,jul_2022,ago_2022,sep_2022,oct_2022,nov_2022,dic_2022,ene_2023,feb_2023,mar_2023,abr_2023,may_2023,jun_2023,jul_2023,ago_2023,sep_2023,oct_2023,nov_2023,dic_2023,ene_2024,feb_2024,mar_2024,abr_2024
0,1994,03,1994-01-24,1.3,1.8,2.8,-7.5,-5.7,-3.1,-7.2,-5.7,-4.6,-3.6,-3.4,4.0,1.2,0.1,5.7,7.2,4.5,7.6,11.6,11.2,10.5,6.3,7.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="2.2.">
   <!-- Contenido de la celda de destino -->
</div>

<div id="2.3.">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">2.2.</span>
    <span style = "color: dark; font-family: PT Serif Pro Book;">
    Creating horizon dataset step by step
    </span>
    </h2>

<div id="steps-2">
   <!-- Contenido de la celda de destino -->
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <a href="#step-2-1" style="text-decoration: none; color: #006769"> <span style="font-size: 24px; color: rgb(0, 65, 75);">&#10122;</span> Replace decimal values by “t+h” values only in the rows representing a new rung</a>
    <br>
    <a href="#step-2-2" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> Concatenate first 3 columns: year, date, id_ns </a>
    <br>
    <a href="#step-2-3" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> Convert columns to string type</a>
    <br>
    <a href="#step-2-4" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10125;</span> Spreads the "t+h" values over the remaining decimal values </a>
    <br>
    <a href="#step-2-5" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10126;</span> Exporting to excel file </a>
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
   <span>Set key variables to fill the growth_rates with 't+h' values</span>
  </div>

In [9]:
# Call the function to show the h_initial window
h_initial = show_h_initial_window()
print("Selected h_initial:", h_initial)

Selected h_initial: 1


In [10]:
# Call the function to show the start_row window
start_row = show_start_row_window()
print("Selected start_row:", start_row)

Selected start_row: 0


In [11]:
# Define the mapping of frequencies to h_counter values
frequency_mapping = {
    'monthly': 1,
    'quarterly': 3,
    'annual': 12
}

# Get the appropriate h_counter value based on the selected frequency
h_counter = frequency_mapping.get(frequency)
print("Selected h_counter:", h_counter)

Selected h_counter: 1


<div id="step-2-1">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10122;</span> <span>Replace decimal values by “t+h” values only in the rows representing a new rung</span>
  </div>

In [12]:
horizon = replace_horizon(growth_rates_df.iloc[:, 3:], start_row, h_initial, h_counter)
horizon_df = horizon.copy()
horizon_df.head(10)

,ene_1992,feb_1992,mar_1992,abr_1992,may_1992,jun_1992,jul_1992,ago_1992,sep_1992,oct_1992,nov_1992,dic_1992,ene_1993,feb_1993,mar_1993,abr_1993,may_1993,jun_1993,jul_1993,ago_1993,sep_1993,oct_1993,nov_1993,dic_1993,ene_1994,feb_1994,mar_1994,abr_1994,may_1994,jun_1994,jul_1994,ago_1994,sep_1994,oct_1994,nov_1994,dic_1994,ene_1995,feb_1995,mar_1995,abr_1995,may_1995,jun_1995,jul_1995,ago_1995,sep_1995,oct_1995,nov_1995,dic_1995,ene_1996,feb_1996,mar_1996,abr_1996,may_1996,jun_1996,jul_1996,ago_1996,sep_1996,oct_1996,nov_1996,dic_1996,ene_1997,feb_1997,mar_1997,abr_1997,may_1997,jun_1997,jul_1997,ago_1997,sep_1997,oct_1997,nov_1997,dic_1997,ene_1998,feb_1998,mar_1998,abr_1998,may_1998,jun_1998,jul_1998,ago_1998,sep_1998,oct_1998,nov_1998,dic_1998,ene_1999,feb_1999,mar_1999,abr_1999,may_1999,jun_1999,jul_1999,ago_1999,sep_1999,oct_1999,nov_1999,dic_1999,ene_2000,feb_2000,mar_2000,abr_2000,may_2000,jun_2000,jul_2000,ago_2000,sep_2000,oct_2000,nov_2000,dic_2000,ene_2001,feb_2001,mar_2001,abr_2001,may_2001,jun_2001,jul_2001,ago_2001,sep_2001,oct_2001,nov_2001,dic_2001,ene_2002,feb_2002,mar_2002,abr_2002,may_2002,jun_2002,jul_2002,ago_2002,sep_2002,oct_2002,nov_2002,dic_2002,ene_2003,feb_2003,mar_2003,abr_2003,may_2003,jun_2003,jul_2003,ago_2003,sep_2003,oct_2003,nov_2003,dic_2003,ene_2004,feb_2004,mar_2004,abr_2004,may_2004,jun_2004,jul_2004,ago_2004,sep_2004,oct_2004,nov_2004,dic_2004,ene_2005,feb_2005,mar_2005,abr_2005,may_2005,jun_2005,jul_2005,ago_2005,sep_2005,oct_2005,nov_2005,dic_2005,ene_2006,feb_2006,mar_2006,abr_2006,may_2006,jun_2006,jul_2006,ago_2006,sep_2006,oct_2006,nov_2006,dic_2006,ene_2007,feb_2007,mar_2007,abr_2007,may_2007,jun_2007,jul_2007,ago_2007,sep_2007,oct_2007,nov_2007,dic_2007,ene_2008,feb_2008,mar_2008,abr_2008,may_2008,jun_2008,jul_2008,ago_2008,sep_2008,oct_2008,nov_2008,dic_2008,ene_2009,feb_2009,mar_2009,abr_2009,may_2009,jun_2009,jul_2009,ago_2009,sep_2009,oct_2009,nov_2009,dic_2009,ene_2010,feb_2010,mar_2010,abr_2010,may_2010,jun_2010,jul_2010,ago_2010,sep_2010,oct_2010,nov_2010,dic_2010,ene_2011,feb_2011,mar_2011,abr_2011,may_2011,jun_2011,jul_2011,ago_2011,sep_2011,oct_2011,nov_2011,dic_2011,ene_2012,feb_2012,mar_2012,abr_2012,may_2012,jun_2012,jul_2012,ago_2012,sep_2012,oct_2012,nov_2012,dic_2012,ene_2013,feb_2013,mar_2013,abr_2013,may_2013,jun_2013,jul_2013,ago_2013,sep_2013,oct_2013,nov_2013,dic_2013,ene_2014,feb_2014,mar_2014,abr_2014,may_2014,jun_2014,jul_2014,ago_2014,sep_2014,oct_2014,nov_2014,dic_2014,ene_2015,feb_2015,mar_2015,abr_2015,may_2015,jun_2015,jul_2015,ago_2015,sep_2015,oct_2015,nov_2015,dic_2015,ene_2016,feb_2016,mar_2016,abr_2016,may_2016,jun_2016,jul_2016,ago_2016,sep_2016,oct_2016,nov_2016,dic_2016,ene_2017,feb_2017,mar_2017,abr_2017,may_2017,jun_2017,jul_2017,ago_2017,sep_2017,oct_2017,nov_2017,dic_2017,ene_2018,feb_2018,mar_2018,abr_2018,may_2018,jun_2018,jul_2018,ago_2018,sep_2018,oct_2018,nov_2018,dic_2018,ene_2019,feb_2019,mar_2019,abr_2019,may_2019,jun_2019,jul_2019,ago_2019,sep_2019,oct_2019,nov_2019,dic_2019,ene_2020,feb_2020,mar_2020,abr_2020,may_2020,jun_2020,jul_2020,ago_2020,sep_2020,oct_2020,nov_2020,dic_2020,ene_2021,feb_2021,mar_2021,abr_2021,may_2021,jun_2021,jul_2021,ago_2021,sep_2021,oct_2021,nov_2021,dic_2021,ene_2022,feb_2022,mar_2022,abr_2022,may_2022,jun_2022,jul_2022,ago_2022,sep_2022,oct_2022,nov_2022,dic_2022,ene_2023,feb_2023,mar_2023,abr_2023,may_2023,jun_2023,jul_2023,ago_2023,sep_2023,oct_2023,nov_2023,dic_2023,ene_2024,feb_2024,mar_2024,abr_2024
0,t+23,t+22,t+21,t+20,t+19,t+18,t+17,t+16,t+15,t+14,t+13,t+12,t+11,t+10,t+9,t+8,t+7,t+6,t+5,t+4,t+3,t+2,t+1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

<div id="step-2-2">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> <span>Concatenate first 3 columns: year, date, id_ns</span>
  </div>

In [13]:
# Get the first three columns of the original DataFrame
first_3_columns = growth_rates_df.iloc[:, :3]

# Concatenate the first three columns with h_{sector}_{frequency}_growth_rates
horizon_df = pd.concat([first_3_columns, horizon_df], axis=1)
horizon_df.head(20)

,year,id_ns,date,ene_1992,feb_1992,mar_1992,abr_1992,may_1992,jun_1992,jul_1992,ago_1992,sep_1992,oct_1992,nov_1992,dic_1992,ene_1993,feb_1993,mar_1993,abr_1993,may_1993,jun_1993,jul_1993,ago_1993,sep_1993,oct_1993,nov_1993,dic_1993,ene_1994,feb_1994,mar_1994,abr_1994,may_1994,jun_1994,jul_1994,ago_1994,sep_1994,oct_1994,nov_1994,dic_1994,ene_1995,feb_1995,mar_1995,abr_1995,may_1995,jun_1995,jul_1995,ago_1995,sep_1995,oct_1995,nov_1995,dic_1995,ene_1996,feb_1996,mar_1996,abr_1996,may_1996,jun_1996,jul_1996,ago_1996,sep_1996,oct_1996,nov_1996,dic_1996,ene_1997,feb_1997,mar_1997,abr_1997,may_1997,jun_1997,jul_1997,ago_1997,sep_1997,oct_1997,nov_1997,dic_1997,ene_1998,feb_1998,mar_1998,abr_1998,may_1998,jun_1998,jul_1998,ago_1998,sep_1998,oct_1998,nov_1998,dic_1998,ene_1999,feb_1999,mar_1999,abr_1999,may_1999,jun_1999,jul_1999,ago_1999,sep_1999,oct_1999,nov_1999,dic_1999,ene_2000,feb_2000,mar_2000,abr_2000,may_2000,jun_2000,jul_2000,ago_2000,sep_2000,oct_2000,nov_2000,dic_2000,ene_2001,feb_2001,mar_2001,abr_2001,may_2001,jun_2001,jul_2001,ago_2001,sep_2001,oct_2001,nov_2001,dic_2001,ene_2002,feb_2002,mar_2002,abr_2002,may_2002,jun_2002,jul_2002,ago_2002,sep_2002,oct_2002,nov_2002,dic_2002,ene_2003,feb_2003,mar_2003,abr_2003,may_2003,jun_2003,jul_2003,ago_2003,sep_2003,oct_2003,nov_2003,dic_2003,ene_2004,feb_2004,mar_2004,abr_2004,may_2004,jun_2004,jul_2004,ago_2004,sep_2004,oct_2004,nov_2004,dic_2004,ene_2005,feb_2005,mar_2005,abr_2005,may_2005,jun_2005,jul_2005,ago_2005,sep_2005,oct_2005,nov_2005,dic_2005,ene_2006,feb_2006,mar_2006,abr_2006,may_2006,jun_2006,jul_2006,ago_2006,sep_2006,oct_2006,nov_2006,dic_2006,ene_2007,feb_2007,mar_2007,abr_2007,may_2007,jun_2007,jul_2007,ago_2007,sep_2007,oct_2007,nov_2007,dic_2007,ene_2008,feb_2008,mar_2008,abr_2008,may_2008,jun_2008,jul_2008,ago_2008,sep_2008,oct_2008,nov_2008,dic_2008,ene_2009,feb_2009,mar_2009,abr_2009,may_2009,jun_2009,jul_2009,ago_2009,sep_2009,oct_2009,nov_2009,dic_2009,ene_2010,feb_2010,mar_2010,abr_2010,may_2010,jun_2010,jul_2010,ago_2010,sep_2010,oct_2010,nov_2010,dic_2010,ene_2011,feb_2011,mar_2011,abr_2011,may_2011,jun_2011,jul_2011,ago_2011,sep_2011,oct_2011,nov_2011,dic_2011,ene_2012,feb_2012,mar_2012,abr_2012,may_2012,jun_2012,jul_2012,ago_2012,sep_2012,oct_2012,nov_2012,dic_2012,ene_2013,feb_2013,mar_2013,abr_2013,may_2013,jun_2013,jul_2013,ago_2013,sep_2013,oct_2013,nov_2013,dic_2013,ene_2014,feb_2014,mar_2014,abr_2014,may_2014,jun_2014,jul_2014,ago_2014,sep_2014,oct_2014,nov_2014,dic_2014,ene_2015,feb_2015,mar_2015,abr_2015,may_2015,jun_2015,jul_2015,ago_2015,sep_2015,oct_2015,nov_2015,dic_2015,ene_2016,feb_2016,mar_2016,abr_2016,may_2016,jun_2016,jul_2016,ago_2016,sep_2016,oct_2016,nov_2016,dic_2016,ene_2017,feb_2017,mar_2017,abr_2017,may_2017,jun_2017,jul_2017,ago_2017,sep_2017,oct_2017,nov_2017,dic_2017,ene_2018,feb_2018,mar_2018,abr_2018,may_2018,jun_2018,jul_2018,ago_2018,sep_2018,oct_2018,nov_2018,dic_2018,ene_2019,feb_2019,mar_2019,abr_2019,may_2019,jun_2019,jul_2019,ago_2019,sep_2019,oct_2019,nov_2019,dic_2019,ene_2020,feb_2020,mar_2020,abr_2020,may_2020,jun_2020,jul_2020,ago_2020,sep_2020,oct_2020,nov_2020,dic_2020,ene_2021,feb_2021,mar_2021,abr_2021,may_2021,jun_2021,jul_2021,ago_2021,sep_2021,oct_2021,nov_2021,dic_2021,ene_2022,feb_2022,mar_2022,abr_2022,may_2022,jun_2022,jul_2022,ago_2022,sep_2022,oct_2022,nov_2022,dic_2022,ene_2023,feb_2023,mar_2023,abr_2023,may_2023,jun_2023,jul_2023,ago_2023,sep_2023,oct_2023,nov_2023,dic_2023,ene_2024,feb_2024,mar_2024,abr_2024
0,1994,03,1994-01-24,t+23,t+22,t+21,t+20,t+19,t+18,t+17,t+16,t+15,t+14,t+13,t+12,t+11,t+10,t+9,t+8,t+7,t+6,t+5,t+4,t+3,t+2,t+1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

<div id="step-2-3">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> <span>Convert columns to string type</span>
  </div>

In [14]:
horizon_df = columns_str(horizon_df)
horizon_df.head(20)

,year,id_ns,date,ene_1992,feb_1992,mar_1992,abr_1992,may_1992,jun_1992,jul_1992,ago_1992,sep_1992,oct_1992,nov_1992,dic_1992,ene_1993,feb_1993,mar_1993,abr_1993,may_1993,jun_1993,jul_1993,ago_1993,sep_1993,oct_1993,nov_1993,dic_1993,ene_1994,feb_1994,mar_1994,abr_1994,may_1994,jun_1994,jul_1994,ago_1994,sep_1994,oct_1994,nov_1994,dic_1994,ene_1995,feb_1995,mar_1995,abr_1995,may_1995,jun_1995,jul_1995,ago_1995,sep_1995,oct_1995,nov_1995,dic_1995,ene_1996,feb_1996,mar_1996,abr_1996,may_1996,jun_1996,jul_1996,ago_1996,sep_1996,oct_1996,nov_1996,dic_1996,ene_1997,feb_1997,mar_1997,abr_1997,may_1997,jun_1997,jul_1997,ago_1997,sep_1997,oct_1997,nov_1997,dic_1997,ene_1998,feb_1998,mar_1998,abr_1998,may_1998,jun_1998,jul_1998,ago_1998,sep_1998,oct_1998,nov_1998,dic_1998,ene_1999,feb_1999,mar_1999,abr_1999,may_1999,jun_1999,jul_1999,ago_1999,sep_1999,oct_1999,nov_1999,dic_1999,ene_2000,feb_2000,mar_2000,abr_2000,may_2000,jun_2000,jul_2000,ago_2000,sep_2000,oct_2000,nov_2000,dic_2000,ene_2001,feb_2001,mar_2001,abr_2001,may_2001,jun_2001,jul_2001,ago_2001,sep_2001,oct_2001,nov_2001,dic_2001,ene_2002,feb_2002,mar_2002,abr_2002,may_2002,jun_2002,jul_2002,ago_2002,sep_2002,oct_2002,nov_2002,dic_2002,ene_2003,feb_2003,mar_2003,abr_2003,may_2003,jun_2003,jul_2003,ago_2003,sep_2003,oct_2003,nov_2003,dic_2003,ene_2004,feb_2004,mar_2004,abr_2004,may_2004,jun_2004,jul_2004,ago_2004,sep_2004,oct_2004,nov_2004,dic_2004,ene_2005,feb_2005,mar_2005,abr_2005,may_2005,jun_2005,jul_2005,ago_2005,sep_2005,oct_2005,nov_2005,dic_2005,ene_2006,feb_2006,mar_2006,abr_2006,may_2006,jun_2006,jul_2006,ago_2006,sep_2006,oct_2006,nov_2006,dic_2006,ene_2007,feb_2007,mar_2007,abr_2007,may_2007,jun_2007,jul_2007,ago_2007,sep_2007,oct_2007,nov_2007,dic_2007,ene_2008,feb_2008,mar_2008,abr_2008,may_2008,jun_2008,jul_2008,ago_2008,sep_2008,oct_2008,nov_2008,dic_2008,ene_2009,feb_2009,mar_2009,abr_2009,may_2009,jun_2009,jul_2009,ago_2009,sep_2009,oct_2009,nov_2009,dic_2009,ene_2010,feb_2010,mar_2010,abr_2010,may_2010,jun_2010,jul_2010,ago_2010,sep_2010,oct_2010,nov_2010,dic_2010,ene_2011,feb_2011,mar_2011,abr_2011,may_2011,jun_2011,jul_2011,ago_2011,sep_2011,oct_2011,nov_2011,dic_2011,ene_2012,feb_2012,mar_2012,abr_2012,may_2012,jun_2012,jul_2012,ago_2012,sep_2012,oct_2012,nov_2012,dic_2012,ene_2013,feb_2013,mar_2013,abr_2013,may_2013,jun_2013,jul_2013,ago_2013,sep_2013,oct_2013,nov_2013,dic_2013,ene_2014,feb_2014,mar_2014,abr_2014,may_2014,jun_2014,jul_2014,ago_2014,sep_2014,oct_2014,nov_2014,dic_2014,ene_2015,feb_2015,mar_2015,abr_2015,may_2015,jun_2015,jul_2015,ago_2015,sep_2015,oct_2015,nov_2015,dic_2015,ene_2016,feb_2016,mar_2016,abr_2016,may_2016,jun_2016,jul_2016,ago_2016,sep_2016,oct_2016,nov_2016,dic_2016,ene_2017,feb_2017,mar_2017,abr_2017,may_2017,jun_2017,jul_2017,ago_2017,sep_2017,oct_2017,nov_2017,dic_2017,ene_2018,feb_2018,mar_2018,abr_2018,may_2018,jun_2018,jul_2018,ago_2018,sep_2018,oct_2018,nov_2018,dic_2018,ene_2019,feb_2019,mar_2019,abr_2019,may_2019,jun_2019,jul_2019,ago_2019,sep_2019,oct_2019,nov_2019,dic_2019,ene_2020,feb_2020,mar_2020,abr_2020,may_2020,jun_2020,jul_2020,ago_2020,sep_2020,oct_2020,nov_2020,dic_2020,ene_2021,feb_2021,mar_2021,abr_2021,may_2021,jun_2021,jul_2021,ago_2021,sep_2021,oct_2021,nov_2021,dic_2021,ene_2022,feb_2022,mar_2022,abr_2022,may_2022,jun_2022,jul_2022,ago_2022,sep_2022,oct_2022,nov_2022,dic_2022,ene_2023,feb_2023,mar_2023,abr_2023,may_2023,jun_2023,jul_2023,ago_2023,sep_2023,oct_2023,nov_2023,dic_2023,ene_2024,feb_2024,mar_2024,abr_2024
0,1994,03,1994-01-24,t+23,t+22,t+21,t+20,t+19,t+18,t+17,t+16,t+15,t+14,t+13,t+12,t+11,t+10,t+9,t+8,t+7,t+6,t+5,t+4,t+3,t+2,t+1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

<div id="step-2-4">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10125;</span> Spreads the "t+h" values over the remaining decimal values
  </div>

In [15]:
horizon_df = replace_horizon_1(horizon_df)
horizon_df.head(20)

,year,id_ns,date,ene_1992,feb_1992,mar_1992,abr_1992,may_1992,jun_1992,jul_1992,ago_1992,sep_1992,oct_1992,nov_1992,dic_1992,ene_1993,feb_1993,mar_1993,abr_1993,may_1993,jun_1993,jul_1993,ago_1993,sep_1993,oct_1993,nov_1993,dic_1993,ene_1994,feb_1994,mar_1994,abr_1994,may_1994,jun_1994,jul_1994,ago_1994,sep_1994,oct_1994,nov_1994,dic_1994,ene_1995,feb_1995,mar_1995,abr_1995,may_1995,jun_1995,jul_1995,ago_1995,sep_1995,oct_1995,nov_1995,dic_1995,ene_1996,feb_1996,mar_1996,abr_1996,may_1996,jun_1996,jul_1996,ago_1996,sep_1996,oct_1996,nov_1996,dic_1996,ene_1997,feb_1997,mar_1997,abr_1997,may_1997,jun_1997,jul_1997,ago_1997,sep_1997,oct_1997,nov_1997,dic_1997,ene_1998,feb_1998,mar_1998,abr_1998,may_1998,jun_1998,jul_1998,ago_1998,sep_1998,oct_1998,nov_1998,dic_1998,ene_1999,feb_1999,mar_1999,abr_1999,may_1999,jun_1999,jul_1999,ago_1999,sep_1999,oct_1999,nov_1999,dic_1999,ene_2000,feb_2000,mar_2000,abr_2000,may_2000,jun_2000,jul_2000,ago_2000,sep_2000,oct_2000,nov_2000,dic_2000,ene_2001,feb_2001,mar_2001,abr_2001,may_2001,jun_2001,jul_2001,ago_2001,sep_2001,oct_2001,nov_2001,dic_2001,ene_2002,feb_2002,mar_2002,abr_2002,may_2002,jun_2002,jul_2002,ago_2002,sep_2002,oct_2002,nov_2002,dic_2002,ene_2003,feb_2003,mar_2003,abr_2003,may_2003,jun_2003,jul_2003,ago_2003,sep_2003,oct_2003,nov_2003,dic_2003,ene_2004,feb_2004,mar_2004,abr_2004,may_2004,jun_2004,jul_2004,ago_2004,sep_2004,oct_2004,nov_2004,dic_2004,ene_2005,feb_2005,mar_2005,abr_2005,may_2005,jun_2005,jul_2005,ago_2005,sep_2005,oct_2005,nov_2005,dic_2005,ene_2006,feb_2006,mar_2006,abr_2006,may_2006,jun_2006,jul_2006,ago_2006,sep_2006,oct_2006,nov_2006,dic_2006,ene_2007,feb_2007,mar_2007,abr_2007,may_2007,jun_2007,jul_2007,ago_2007,sep_2007,oct_2007,nov_2007,dic_2007,ene_2008,feb_2008,mar_2008,abr_2008,may_2008,jun_2008,jul_2008,ago_2008,sep_2008,oct_2008,nov_2008,dic_2008,ene_2009,feb_2009,mar_2009,abr_2009,may_2009,jun_2009,jul_2009,ago_2009,sep_2009,oct_2009,nov_2009,dic_2009,ene_2010,feb_2010,mar_2010,abr_2010,may_2010,jun_2010,jul_2010,ago_2010,sep_2010,oct_2010,nov_2010,dic_2010,ene_2011,feb_2011,mar_2011,abr_2011,may_2011,jun_2011,jul_2011,ago_2011,sep_2011,oct_2011,nov_2011,dic_2011,ene_2012,feb_2012,mar_2012,abr_2012,may_2012,jun_2012,jul_2012,ago_2012,sep_2012,oct_2012,nov_2012,dic_2012,ene_2013,feb_2013,mar_2013,abr_2013,may_2013,jun_2013,jul_2013,ago_2013,sep_2013,oct_2013,nov_2013,dic_2013,ene_2014,feb_2014,mar_2014,abr_2014,may_2014,jun_2014,jul_2014,ago_2014,sep_2014,oct_2014,nov_2014,dic_2014,ene_2015,feb_2015,mar_2015,abr_2015,may_2015,jun_2015,jul_2015,ago_2015,sep_2015,oct_2015,nov_2015,dic_2015,ene_2016,feb_2016,mar_2016,abr_2016,may_2016,jun_2016,jul_2016,ago_2016,sep_2016,oct_2016,nov_2016,dic_2016,ene_2017,feb_2017,mar_2017,abr_2017,may_2017,jun_2017,jul_2017,ago_2017,sep_2017,oct_2017,nov_2017,dic_2017,ene_2018,feb_2018,mar_2018,abr_2018,may_2018,jun_2018,jul_2018,ago_2018,sep_2018,oct_2018,nov_2018,dic_2018,ene_2019,feb_2019,mar_2019,abr_2019,may_2019,jun_2019,jul_2019,ago_2019,sep_2019,oct_2019,nov_2019,dic_2019,ene_2020,feb_2020,mar_2020,abr_2020,may_2020,jun_2020,jul_2020,ago_2020,sep_2020,oct_2020,nov_2020,dic_2020,ene_2021,feb_2021,mar_2021,abr_2021,may_2021,jun_2021,jul_2021,ago_2021,sep_2021,oct_2021,nov_2021,dic_2021,ene_2022,feb_2022,mar_2022,abr_2022,may_2022,jun_2022,jul_2022,ago_2022,sep_2022,oct_2022,nov_2022,dic_2022,ene_2023,feb_2023,mar_2023,abr_2023,may_2023,jun_2023,jul_2023,ago_2023,sep_2023,oct_2023,nov_2023,dic_2023,ene_2024,feb_2024,mar_2024,abr_2024
0,1994,03,1994-01-24,t+23,t+22,t+21,t+20,t+19,t+18,t+17,t+16,t+15,t+14,t+13,t+12,t+11,t+10,t+9,t+8,t+7,t+6,t+5,t+4,t+3,t+2,t+1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

<div id="step-2-5">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10126;</span> Exporting to excel file
  </div>

In [ ]:
#def export_to_excel(df, filename):
#    # Exportar el DataFrame como un archivo Excel
#    df.to_excel(filename, index=False)

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-2" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-2" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="3">
   <!-- Contenido de la celda de destino -->
</div>

<h1><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">3.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Create growth rates by horizon dataset</span></h1>

<div id="steps-3">
   <!-- Contenido de la celda de destino -->
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <a href="#st-3-1" style="text-decoration: none; color: #006769"> <span style="font-size: 24px; color: rgb(0, 65, 75);">&#10122;</span> Generating a dictionary with the row indices and their t+h values</a>
    <br>
    <a href="#st-3-2" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> Colapse growth rates by horizon ('t+h') </a>
    <br>
</div>

<div id="st-3-1">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10122;</span> <span>Generating a dictionary with the row indices and their t+h values</span>
  </div>

In [19]:
dictionary = get_last_index_h(horizon_df)

<div id="st-3-2">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> <span>Colapse growth rates by horizon ('t+h')  </span>
  </div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Note that <code>'t'</code> is the date of the first announcement, while <code>'t+h'</code> is 'h' months after the first announcement.
    <div/>

In [20]:
filtered_h_df = filter_df_by_indices(growth_rates_df, dictionary)
filtered_h_df

,horizon,year_2010,year_2011,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018,year_2019,year_2020,year_2021,year_2022,year_2023
0,t,8.8,6.9,6.3,5.0,2.4,3.3,3.9,2.5,4.0,2.2,-11.1,13.3,2.7,-0.6
1,t+1,8.8,6.9,6.3,5.0,2.4,3.3,3.9,2.5,4.0,2.2,-11.1,13.3,2.7,-0.6
2,t+2,8.8,6.9,6.3,5.0,2.4,3.3,3.9,2.5,4.0,2.2,-11.1,13.3,2.7,-0.6
3,t+3,8.8,6.9,6.3,5.8,2.4,3.3,3.9,2.5,4.0,2.2,-11.1,13.5,2.7,-0.6
4,t+4,8.8,6.9,6.3,5.8,2.4,3.3,3.9,2.5,4.0,2.2,-11.1,13.5,2.7,-0.6
5,t+5,8.8,6.9,6.3,5.8,2.4,3.3,3.9,2.5,4.0,2.2,-11.1,13.5,2.7,NaN
6,t+6,8.8,6.9,6.3,5.8,2.4,3.3,4.0,2.5,4.0,2.2,-11.0,13.6,2.7,NaN
7,t+7,8.8,6.9,6.3,5.8,2.4,3.3,4.0,2.5,4.0,2.2,-11.0,13.6,2.7,NaN
8,t+8,8.8,6.9,6.3,5.8,2.4,3.3,4.0,2.5,4.0,2.2,-11.0,13.6,2.7,NaN
9,t+9,NaN,6.9,6.3,5.8,2.4,3.3,4.0,2.5,4.0,2.2,-11.0,13.6,2.7,NaN


<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-1" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-3" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="4">
   <!-- Contenido de la celda de destino -->
</div>

<h1><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">4.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Create vintages and releases datasets</span></h1>

<div id="steps-4">
   <!-- Contenido de la celda de destino -->
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <a href="#st-4-1" style="text-decoration: none; color: #006769"> <span style="font-size: 24px; color: rgb(0, 65, 75);">&#10122;</span> Create vintages</a>
    <br>
    <a href="#st-4-2" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> Clean-up vintages dataframe </a>
    <br>
    <a href="#st-4-3" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> Create releases </a>
    <br>
</div>

<div id="st-4-1">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10122;</span> <span>Create vintages</span>
  </div>

In [21]:
vintages = create_vintages(growth_rates)
vintages

aux,vintages_date,ns_2013m01,ns_2013m02,ns_2013m03,ns_2013m04,ns_2013m05,ns_2013m06,ns_2013m07,ns_2013m08,ns_2013m09,ns_2013m10,ns_2013m11,ns_2013m12,ns_2014m01,ns_2014m02,ns_2014m03,ns_2014m04,ns_2014m05,ns_2014m06,ns_2014m07,ns_2014m08,ns_2014m09,ns_2014m10,ns_2014m11,ns_2014m12,ns_2015m01,ns_2015m02,ns_2015m03,ns_2015m04,ns_2015m05,ns_2015m06,ns_2015m07,ns_2015m08,ns_2015m09,ns_2015m10,ns_2015m11,ns_2015m12,ns_2016m01,ns_2016m02,ns_2016m03,ns_2016m04,ns_2016m05,ns_2016m06,ns_2016m07,ns_2016m08,ns_2016m09,ns_2016m10,ns_2016m11,ns_2016m12,ns_2017m01,ns_2017m02,ns_2017m03,ns_2017m04,ns_2017m05,ns_2017m06,ns_2017m07,ns_2017m08,ns_2017m09,ns_2017m10,ns_2017m11,ns_2017m12,ns_2018m01,ns_2018m02,ns_2018m03,ns_2018m04,ns_2018m05,ns_2018m06,ns_2018m07,ns_2018m08,ns_2018m09,ns_2018m10,ns_2018m11,ns_2018m12,ns_2019m01,ns_2019m02,ns_2019m03,ns_2019m04,ns_2019m05,ns_2019m06,ns_2019m07,ns_2019m08,ns_2019m09,ns_2019m10,ns_2019m11,ns_2019m12,ns_2020m01,ns_2020m02,ns_2020m03,ns_2020m04,ns_2020m05,ns_2020m06,ns_2020m07,ns_2020m08,ns_2020m09,ns_2020m10,ns_2020m11,ns_2020m12,ns_2021m01,ns_2021m02,ns_2021m03,ns_2021m04,ns_2021m05,ns_2021m06,ns_2021m07,ns_2021m08,ns_2021m09,ns_2021m10,ns_2021m11,ns_2021m12,ns_2022m01,ns_2022m02,ns_2022m03,ns_2022m04,ns_2022m05,ns_2022m06,ns_2022m07,ns_2022m08,ns_2022m09,ns_2022m10,ns_2022m11,ns_2022m12,ns_2023m01,ns_2023m02,ns_2023m03,ns_2023m04,ns_2023m05,ns_2023m06,ns_2023m07,ns_2023m08,ns_2023m09,ns_2023m10,ns_2023m11,ns_2023m12,ns_2024m01,ns_2024m02,ns_2024m03,ns_2024m04,ns_2024m05,ns_2024m06
0,year_2010,8.8,8.8,8.8,8.8,8.8,NaN,NaN,8.8,8.8,8.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,year_2011,6.9,6.9,6.9,6.9,6.9,6.9,6.9,6.9,6.9,6.9,6.9,6.9,6.9,6.9,6.9,6.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,year_2012,NaN,6.3,6.3,6.3,6.3,6.3,6.3,6.3,6.3,6.3,6.3,6.3,6.3,6.3,6.3,6.3,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,year_2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,5.0,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.9,5.9,5.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,year_2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,

<div id="st-4-2">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> <span>Clean-up vintages dataframe</span>
  </div>

In [22]:
# Call the appropriate function
if frequency == 'monthly':
    vintages_df = process_monthly(vintages)
elif frequency == 'quarterly':
    vintages_df = process_quarterly(vintages)
elif frequency == 'annual':
    vintages_df = process_annual(vintages)

# Display the result
vintages_df

aux,vintages_date,ns_2013m01,ns_2013m02,ns_2013m03,ns_2013m04,ns_2013m05,ns_2013m06,ns_2013m07,ns_2013m08,ns_2013m09,ns_2013m10,ns_2013m11,ns_2013m12,ns_2014m01,ns_2014m02,ns_2014m03,ns_2014m04,ns_2014m05,ns_2014m06,ns_2014m07,ns_2014m08,ns_2014m09,ns_2014m10,ns_2014m11,ns_2014m12,ns_2015m01,ns_2015m02,ns_2015m03,ns_2015m04,ns_2015m05,ns_2015m06,ns_2015m07,ns_2015m08,ns_2015m09,ns_2015m10,ns_2015m11,ns_2015m12,ns_2016m01,ns_2016m02,ns_2016m03,ns_2016m04,ns_2016m05,ns_2016m06,ns_2016m07,ns_2016m08,ns_2016m09,ns_2016m10,ns_2016m11,ns_2016m12,ns_2017m01,ns_2017m02,ns_2017m03,ns_2017m04,ns_2017m05,ns_2017m06,ns_2017m07,ns_2017m08,ns_2017m09,ns_2017m10,ns_2017m11,ns_2017m12,ns_2018m01,ns_2018m02,ns_2018m03,ns_2018m04,ns_2018m05,ns_2018m06,ns_2018m07,ns_2018m08,ns_2018m09,ns_2018m10,ns_2018m11,ns_2018m12,ns_2019m01,ns_2019m02,ns_2019m03,ns_2019m04,ns_2019m05,ns_2019m06,ns_2019m07,ns_2019m08,ns_2019m09,ns_2019m10,ns_2019m11,ns_2019m12,ns_2020m01,ns_2020m02,ns_2020m03,ns_2020m04,ns_2020m05,ns_2020m06,ns_2020m07,ns_2020m08,ns_2020m09,ns_2020m10,ns_2020m11,ns_2020m12,ns_2021m01,ns_2021m02,ns_2021m03,ns_2021m04,ns_2021m05,ns_2021m06,ns_2021m07,ns_2021m08,ns_2021m09,ns_2021m10,ns_2021m11,ns_2021m12,ns_2022m01,ns_2022m02,ns_2022m03,ns_2022m04,ns_2022m05,ns_2022m06,ns_2022m07,ns_2022m08,ns_2022m09,ns_2022m10,ns_2022m11,ns_2022m12,ns_2023m01,ns_2023m02,ns_2023m03,ns_2023m04,ns_2023m05,ns_2023m06,ns_2023m07,ns_2023m08,ns_2023m09,ns_2023m10,ns_2023m11,ns_2023m12,ns_2024m01,ns_2024m02,ns_2024m03,ns_2024m04,ns_2024m05,ns_2024m06
0,2010-01-01,8.8,8.8,8.8,8.8,8.8,NaN,NaN,8.8,8.8,8.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011-01-01,6.9,6.9,6.9,6.9,6.9,6.9,6.9,6.9,6.9,6.9,6.9,6.9,6.9,6.9,6.9,6.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012-01-01,NaN,6.3,6.3,6.3,6.3,6.3,6.3,6.3,6.3,6.3,6.3,6.3,6.3,6.3,6.3,6.3,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,5.0,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.9,5.9,5.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4

<div id="st-4-3">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> <span>Create releases</span>
  </div>

In [23]:
releases_df = create_releases(vintages_df, sector)
releases_df

aux,vintages_date,gdp_release_1,gdp_release_2,gdp_release_3,gdp_release_4,gdp_release_5,gdp_release_6,gdp_release_7,gdp_release_8,gdp_release_9,gdp_release_10,gdp_release_11,gdp_release_12,gdp_release_13,gdp_release_14,gdp_release_15,gdp_release_16,gdp_release_17,gdp_release_18,gdp_release_19,gdp_release_20,gdp_release_21,gdp_release_22,gdp_release_23,gdp_release_24,gdp_release_25,gdp_release_26,gdp_release_27,gdp_release_28,gdp_release_29,gdp_release_30,gdp_release_31,gdp_release_32,gdp_release_33,gdp_release_34,gdp_release_35,gdp_release_36,gdp_release_37,gdp_release_38,gdp_release_39,gdp_most_recent
0,2010-01-01,8.8,8.8,8.8,8.8,8.8,8.8,8.8,8.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.8
1,2011-01-01,6.9,6.9,6.9,6.9,6.9,6.9,6.9,6.9,6.9,6.9,6.9,6.9,6.9,6.9,6.9,6.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.9
2,2012-01-01,6.3,6.3,6.3,6.3,6.3,6.3,6.3,6.3,6.3,6.3,6.3,6.3,6.3,6.3,6.3,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0
3,2013-01-01,5.0,5.0,5.0,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.8,5.9,5.9,5.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.9
4,2014-01-01,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.4
5,2015-01-01,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.3
6,2016-01-01,3.9,3.9,3.9,3.9,3.9,3.9,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.1,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
7,2017-01-01,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.1,2.1,2.5,NaN,NaN,NaN,NaN,2.5
8,2018-01-01,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0
9,2019-01-01,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2,2.2,NaN,NaN,NaN,2.2


<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-4" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-4" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

#### Check column type

In [24]:
print(releases_df['gdp_release_1'].dtype)

float64


In [25]:
print(releases_df['gdp_release_1'].dtype)

float64


<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="5">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: charter;">5.</span>
    <span style = "color: dark; font-family: charter;">
    Loading to SQL
    </span>
    </h2>

In [29]:
#horizon_df.to_sql(f'{sector}_{frequency}_growth_rates_horizon', engine, index=False, if_exists='replace')
filtered_h_df.to_sql(f'{sector}_{frequency}_h', engine, index=False, if_exists='replace')
vintages_df.to_sql(f'{sector}_{frequency}_vintages', engine, index=False, if_exists='replace')
releases_df.to_sql(f'{sector}_{frequency}_releases', engine, index=False, if_exists='replace')

14

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#step-1-1" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#step-1-1" style="color: rgb(255, 32, 78); text-decoration: none;">Back to <b>select economic sector</b>.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>